In [1]:
import os
import shutil

%cd /content

folder_name = "hindawi_rag_1"

if os.path.exists(folder_name):
    shutil.rmtree(folder_name)
    print(f"{folder_name} deleted!")

repo_url = f"https://github.com/NourMansour911/hindawi_rag_1.git"
os.system(f"git clone -qq {repo_url}")
print(os.getcwd())
!ls
%cd hindawi_rag_1
!ls  
!pip -qqq install -r req.txt

/content
hindawi_rag_1 deleted!
/content
hindawi_rag_1  sample_data
/content/hindawi_rag_1
req.txt  script-1.ipynb


In [2]:
!pip -qqq install PyPDF2 requests

In [3]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("intfloat/e5-base-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [4]:
""" from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",dtype=torch.float16,
    
) """

' from transformers import AutoTokenizer, AutoModelForCausalLM\nimport torch\n\nmodel_name = "mistralai/Mistral-Nemo-Instruct-2407"\ntokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True)\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_name,\n    device_map="auto",dtype=torch.float16,\n    \n) '

In [5]:
""" def generate_text(prompt, max_length=100, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs] """

' def generate_text(prompt, max_length=100, num_return_sequences=1):\n    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)\n    outputs = model.generate(\n        **inputs,\n        max_length=max_length,\n        num_return_sequences=num_return_sequences,\n        do_sample=True,\n        top_k=50,\n        top_p=0.95,\n        temperature=0.7,\n    )\n    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs] '

In [6]:
file_id = "1shdOUK53IGL2yCKdQdgL169st59SPTUw"

In [7]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader

url = f"https://drive.google.com/uc?export=download&id={file_id}"

response = requests.get(url)

reader = PdfReader(BytesIO(response.content))

In [8]:
def extract_text_from_pdf(reader):
    reader = reader
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() + "\n"
    return full_text

In [9]:
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [10]:
import re
text = extract_text_from_pdf(reader)


In [11]:
def clean_pdf_text_keep_structure(text):
    text = re.sub(r"\n\s*\d+\s*\n", "\n", text)

    text = re.sub(r"(?:\n\s*[•]\s*){2,}", "\n", text)

    text = re.sub(r"•\s*\n", "\n", text)

    text = re.sub(r"(\n\s*){2,}", "\n", text)

    return text.strip()

In [12]:
text = clean_pdf_text_keep_structure(text)

In [13]:
print(text)

1. General Overview
Tips Hindawi University (THU) is a premier institution of higher education located in the heart of the Middle
East. Founded in 1963, the university has grown into a globally recognized center for academic excellence
and innovation. With over six decades of educational leadership, THU has produced more than 150,000
graduates who serve in diverse industries and academic circles worldwide.
The university is accredited by the International Commission for Academic Standards and the Ministry of
Higher Education. It operates under the guiding motto: "Knowledge, Integrity, Progress" , and fosters an
environment of research, creativity, and public service.
2. Campus and Facilities
2.1 Main Campus
The main campus is located in the capital city and spans over 300 acres. It houses: - 12 academic buildings -
Central library with over 1 million volumes - Four residence halls - A medical center - Sports complex and
stadium - Innovation and entrepreneurship hub
2.2 Satellite Campus

In [31]:
def chunk_text(text, chunk_size=250, overlap=30):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [32]:
chunks = chunk_text(text)

In [33]:
embeddings = emb_model.encode(chunks, convert_to_numpy=True)

In [34]:
import faiss

def create_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [35]:
def search_index(query,  index, chunks, k=5):
    query_embedding = emb_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [36]:
index = create_faiss_index(embeddings)

In [37]:
questions = ["Where is Tips Hindawi University located?","Does the university offer online programs?",
           "Is there financial aid for international students?","What languages are used for instruction?" ]

In [38]:
""" for q in  questions:
    top_chunk = search_index(q, index, chunks, k=1)
    prompt = f"Answer the next question: {q} by reading the following text:{top_chunk[0]}"
    print(prompt)
    answer = generate_text(prompt,num_return_sequences=1,max_length=1024)
    print(f"{q}?\n\nAnswer:\n",answer)
     """

' for q in  questions:\n    top_chunk = search_index(q, index, chunks, k=1)\n    prompt = f"Answer the next question: {q} by reading the following text:{top_chunk[0]}"\n    print(prompt)\n    answer = generate_text(prompt,num_return_sequences=1,max_length=1024)\n    print(f"{q}?\n\nAnswer:\n",answer)\n     '

In [39]:
OPENROUTER_API_KEY = input("OPENROUTER_API_KEY: ")

In [65]:
model_name = "xiaomi/mimo-v2-flash:free"

In [66]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    model=model_name)

In [72]:
answers = []
selected_chunks = []

In [78]:
from langchain_core.prompts import PromptTemplate

# Explain {subject} like i am 5
string_prompt = PromptTemplate.from_template("""You are an assistant answering questions using the provided context.

If the answer is explicitly stated, answer directly.
If the answer can be reasonably inferred from the context, answer and say "Based on the document".
If the answer is not present at all, say "Not mentioned in the document".

Context:
{context}

Question:
{q}
""")

In [79]:
for q in  questions:
    top_chunk = search_index(q, index, chunks, k=3)
    prompt = string_prompt.format(q=q, context=top_chunk[0] + top_chunk[1] + top_chunk[2])
    selected_chunks.append(top_chunk[0])
    print(prompt+"\n\n\n")
    answers.append(llm.invoke(prompt).content)

You are an assistant answering questions using the provided context.

If the answer is explicitly stated, answer directly.
If the answer can be reasonably inferred from the context, answer and say "Based on the document".
If the answer is not present at all, say "Not mentioned in the document".

Context:
1. General Overview Tips Hindawi University (THU) is a premier institution of higher education located in the heart of the Middle East. Founded in 1963, the university has grown into a globally recognized center for academic excellence and innovation. With over six decades of educational leadership, THU has produced more than 150,000 graduates who serve in diverse industries and academic circles worldwide. The university is accredited by the International Commission for Academic Standards and the Ministry of Higher Education. It operates under the guiding motto: "Knowledge, Integrity, Progress" , and fosters an environment of research, creativity, and public service. 2. Campus and Faci

In [80]:
from IPython.display import display, Markdown

for i,a in enumerate(answers):
    display(Markdown(f"Question: {questions[i]}"))
    display(Markdown(a))
    print("\n\n")
    print("_"*100)

Question: Where is Tips Hindawi University located?

Based on the document, Tips Hindawi University (THU) is located in the heart of the Middle East, specifically in the capital city.




____________________________________________________________________________________________________


Question: Does the university offer online programs?

Not mentioned in the document.

...

Based on the document, yes. 

Section 4.4 states that the university offers Merit Scholarships (25% to 100% tuition coverage), Need-Based Grants, and Research Fellowships for graduate students. Additionally, Section 2.3 lists "Amal Housing Complex" as housing for international students, and Section 5 mentions an "Office of International Programs" that assists them.




____________________________________________________________________________________________________


Question: What languages are used for instruction?

Not mentioned in the document




____________________________________________________________________________________________________


In [81]:
top_chunk

['Law and International Studies Faculty of Computer and Information Sciences Faculty of Architecture and Design 3.2 Sample Undergraduate Programs BSc in Computer Science BA in International Relations BBA in Marketing BEng in Civil Engineering BSc in Nursing 3.3 Graduate Programs MBA (Master of Business Administration) MSc in Data Science MA in Psychology PhD in Mechanical Engineering PhD in Political Science 4. Admissions and Tuition 4.1 Undergraduate Admissions High school GPA of 85% or equivalent English proficiency test (IELTS 6.0 or TOEFL 80) Entrance interview for certain programs 4.2 Graduate Admissions Relevant bachelor’s degree Minimum GPA of 3.0/4.0 Two academic references Statement of purpose 4.3 Tuition Fees (Annual) Undergraduate: $5,000 - $8,000 Graduate: $6,500 - $10,000 4.4 Scholarships Merit Scholarships : 25% to 100% tuition coverage Need-Based Grants Research Fellowships for graduate students 5. Administration President : Dr . Nabil Al-Khatib Vice President of Academi